<center><b>Tensorflow Keras model to train on the GTEx data</b></center>

#### __Aim__: To build a TensorFlow Keras model, train on the available gene expression data to identify the age of the owner of the gene expression.

In [7]:
#Import the required packages

import os
from pathlib import Path
import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import load_model
from sklearn.preprocessing import LabelEncoder
from sklearn import preprocessing

#### Extract the meta information from the data

In [2]:
def GTEx_sample_shrinker(meta,by_col,n=20):
    by=meta[by_col].unique()
    ans=[]
    for i in by:
        nTissue=len(meta[meta[by_col]==i])
        if nTissue<n:
            nn=nTissue
        else:
            nn=n
        ans.append(meta[meta[by_col]==i].sample(nn))
    selectedMeta=pd.concat(ans)
    return selectedMeta

current_dir = os.getcwd() #current directory
data_dir = os.path.join(current_dir, "data")
manifest={"data":"All_Tissue_Site_Details.combined.reads.gct",
          "sample_meta":"GTEx_v7_Annotations_SampleAttributesDS.txt",
          "subject_meta":"GTEx_v7_Annotations_SubjectPhenotypesDS.txt",
           "merged_meta":"merged_meta.tsv"}
meta=pd.read_csv(os.path.join(data_dir, manifest['merged_meta']),sep="\t",dtype={'SMUBRID':object})
y=GTEx_sample_shrinker(meta,'SMTS',20)
y.to_csv(os.path.join(data_dir,"filteredMeta.tsv"),sep="\t",index=False)
y['SAMPID'].to_csv(os.path.join(data_dir,"filteredSAMPID.tsv"),sep="\t",index=False)

#### Identify all the unique tissue types before building the DL model

In [4]:
fileName = os.path.join(data_dir, "merged_meta.tsv")
meta=pd.read_csv(os.path.join(data_dir,manifest['merged_meta']),sep="\t",dtype={'SMUBRID':object,'SEX':object,'DTHHRDY':object})

meta=meta[~(meta['AGE'].isnull())] # removes all samples without age

#Extract only the tissue types with count > 200
counts=pd.DataFrame(meta['SMTS'].value_counts())
df=meta[meta['SMTS'].isin(counts[counts['SMTS'] > 200].index)]

#Identify the unique tissue types
tissue_types = df['SMTS'].unique()

print("Unique tissue types in the GTEx data: ", [t for t in tissue_types])

Unique tissue types in the GTEx data:  ['Adipose Tissue', 'Blood Vessel', 'Brain', 'Breast', 'Blood', 'Skin', 'Colon', 'Esophagus', 'Heart', 'Lung', 'Muscle', 'Nerve', 'Pancreas', 'Stomach', 'Testis', 'Thyroid']


#### Building the keras model

In [5]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import load_model

class keras_model:
    DATA_DIR = "keras_models"
    
    def __init__(self):
        self.early_stopping_monitor = EarlyStopping(patience=3) #Hyperparameter tuning
        
    def construct_model(self, x_train, y_train):
        self.model = tf.keras.models.Sequential() #Sequential model
        self.model.add(tf.keras.layers.Flatten())
        self.model.add(tf.keras.layers.Dense(1024, input_dim=x_train.shape[1], activation=tf.nn.relu))
        self.model.add(tf.keras.layers.Dense(512, activation=tf.nn.relu))
        #self.model.add(tf.keras.layers.Dense(256, activation=tf.nn.relu))
        #self.model.add(tf.keras.layers.Dense(128, activation=tf.nn.relu))
        self.model.add(tf.keras.layers.Dense(y_train.shape[1], activation=tf.nn.softmax))
        #Note: Output layer is designed to hold the number of neurons equivalent to the number of classes of age groups

        self.model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
        
    def model_train(self, x_train, y_train):
        if x_train.shape[0] == y_train.shape[0]: #Checking shape since there are tissues with missing gene expressions
            try:
                print("Training Keras model...")
                
                self.construct_model(x_train, y_train) #Constructing the model
                self.model.fit(x_train, 
                               y_train, 
                               batch_size=32, #Setting batch size for ease of processing in local machines
                               epochs=30, #Maximum of 30 epochs
                               validation_split=0.1, 
                               callbacks=[self.early_stopping_monitor]) #Early stopping hyperparameter
                
                print("Evaluating training accuracy...")
                loss, accuracy = self.model.evaluate(x_train, y_train)
                
                try:
                    #Persisting the model trained for the corresponding tissue type
                    fileName = TISSUE + "_keras_model.h5"
                    filePath = os.path.join(self.DATA_DIR, fileName)
                    self.model.save(filePath)
                    return accuracy, fileName
                except:
                    print("Exception while saving the model.")
                    return accuracy, None
            except:
                print("Exception while processing!")
                return -1, None
        else:
            print("Shape mismatch encountered!")
            return -1, None

#### Train the model for each tissue type

In [8]:
tissue_specific_path = "tissue-specific"

tissue_type = []
tissue_model_persist = []
tissue_model_accuracy = []

for tissue in tissue_types:
    k_model = keras_model()
    TISSUE=tissue
    infiles=os.listdir(tissue_specific_path)
    TISSUE_files=[f for f in infiles if  TISSUE in f]
    for entry in TISSUE_files:
        if "_cpm" in entry: #Identify the file with _cpm suffix; cpm stands for Counts Per Million
            pdd = pd.read_csv(os.path.join(tissue_specific_path,entry), sep='\t')
            
            print("Tissue type: ", TISSUE)
            
            pdd = pdd.drop(pdd.columns[0], axis='columns')

            print(pdd.shape)
            min_max_scaler = preprocessing.MinMaxScaler()
            np_scaled = min_max_scaler.fit_transform(pdd)
            pdd = pd.DataFrame(np_scaled)
            #df_normalized
            print(pdd.shape)
            
            numpy_matrix = pdd.as_matrix()
            #numpy_matrix1 = np.delete(numpy_matrix, 0, axis=1) #Deleting the first column since it contains the gene id and does not play any role in classification
            display(numpy_matrix.shape)
            #break
            #Categorizing the target column and performing one-hot encoding
            tissue_meta=meta[meta['SMTS']==TISSUE]
            encoder = LabelEncoder()
            age_y = tissue_meta['AGE']
            print(age_y.shape)
            encoder.fit(tissue_meta['AGE'])
            encoded_Y = encoder.transform(tissue_meta['AGE'])
            dummy_y = tf.keras.utils.to_categorical(encoded_Y)
            
            #Training the model for the current tissue type
            acc, fileName = k_model.model_train(numpy_matrix, dummy_y)
            if acc != -1:
                try:
                    acc = acc * 100
                    tissue_type.append(TISSUE)
                    tissue_model_persist.append(fileName)
                    tissue_model_accuracy.append(acc)
                    print("Final accuracy:", acc)
                except:
                    print("Error occurred for tissue type: ", TISSUE)
            print("\n")
            break
    #break

Tissue type:  Adipose Tissue
(797, 56202)


C:\Users\suji1\Anaconda3\envs\cs418env\lib\site-packages\sklearn\preprocessing\data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


(797, 56202)


C:\Users\suji1\Anaconda3\envs\cs418env\lib\site-packages\ipykernel_launcher.py:27: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


(797, 56202)

...

C:\Users\suji1\Anaconda3\envs\cs418env\lib\site-packages\sklearn\preprocessing\data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


(446, 56202)


C:\Users\suji1\Anaconda3\envs\cs418env\lib\site-packages\ipykernel_launcher.py:27: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


(446, 56202)

(446,)
Training Keras model...
Train on 401 samples, validate on 45 samples
Epoch 1/30
401/401 [==============================] - 7s 17ms/sample - loss: 9.6065 - acc: 0.3392 - val_loss: 10.3872 - val_acc: 0.3556
Epoch 2/30
401/401 [==============================] - 7s 16ms/sample - loss: 10.6918 - acc: 0.3367 - val_loss: 10.3872 - val_acc: 0.3556
Epoch 3/30
401/401 [==============================] - 6s 15ms/sample - loss: 10.6918 - acc: 0.3367 - val_loss: 10.3872 - val_acc: 0.3556
Epoch 4/30
401/401 [==============================] - 7s 17ms/sample - loss: 10.6918 - acc: 0.3367 - val_loss: 10.3872 - val_acc: 0.3556
Evaluating training accuracy...
446/446 [==============================] - 1s 2ms/sample - loss: 10.6611 - acc: 0.3386
Final accuracy: 33.856502175331116




In [9]:
# from sklearn import preprocessing

# df = pd.read_csv(os.path.join(tissue_specific_path, 'Fallopian Tube_cpm.tsv'), sep='\t')

# #display(df.head())

# #print(df.columns[0])

# df = df.drop(df.columns[0], axis='columns')

# min_max_scaler = preprocessing.MinMaxScaler()
# np_scaled = min_max_scaler.fit_transform(df)
# df_normalized = pd.DataFrame(np_scaled)
# df_normalized

C:\Users\suji1\Anaconda3\envs\cs418env\lib\site-packages\sklearn\preprocessing\data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


,0,1,2,3,4,5,6,7,8,9,...,56192,56193,56194,56195,56196,56197,56198,56199,56200,56201
0,0.549524,0.005246,0.797292,0.265764,0.0,0.273766,0.732698,0.000000,0.198439,0.068498,...,1.000000,0.367008,1.0,0.000000,1.000000,1.000000,1.000000,1.000000,0.230231,1.000000
1,1.000000,0.512791,0.544079,0.000000,0.0,0.000000,1.000000,0.126293,0.050115,0.536488,...,0.708998,0.500899,0.0,0.000000,0.000000,0.000000,0.000000,0.082848,0.000000,0.014275
2,0.000000,0.000000,0.000000,0.366355,0.0,0.377386,0.000000,1.000000,1.000000,1.000000,...,0.000000,0.000000,0.0,0.000000,0.443928,0.497529,0.457159,0.000000,0.158686,0.194009
3,0.334545,0.103648,0.728077,0.485384,0.0,1.000000,0.000000,0.145535,0.774437,0.705069,...,0.237341,0.000000,0.0,0.000000,0.498880,0.447460,0.472846,0.069782,0.210244,0.781234
4,0.499103,0.410866,0.000000,0.000000,0.0,0.000000,0.000000,0.014789,0.264992,0.000000,...,0.229852,1.000000,0.0,1.000000,0.337191,0.309335,0.472481,0.181400,0.156830,0.189845
5,0.000000,1.000000,0.000000,0.384779,0.0,0.000000,0.000000,0.047822,0.000000,0.169561,...,0.007630,0.000000,0.0,0.000000,0.357298,0.373433,0.194012,0.261523,1.000000,0.024991
6,0.000000,0.560312,1.000000,1.000000,0.0,0.686741,0.918984,0.163944,0.346005,0.194229,...,0.999882,0.000000,0.0,0.920636,0.396382,0.247305,0.188485,0.322239,0.288766,0.000000


#### Persist the results of the model in a TSV file for evaluation later

In [11]:
keras_model_pd = pd.DataFrame(
    data = {
        'tissue_type': tissue_type,
        'model_file': tissue_model_persist,
        'model_accuracy': tissue_model_accuracy
    }
)

keras_model_pd = keras_model_pd.sort_values('tissue_type')

keras_model_pd.to_csv("keras_model_results.tsv",sep="\t",index=False)

keras_model_pd

,tissue_type,model_file,model_accuracy
0,Adipose Tissue,Adipose Tissue_keras_model.h5,34.253451
1,Blood Vessel,Blood Vessel_keras_model.h5,17.524645
2,Brain,Brain_keras_model.h5,45.921451
3,Breast,Breast_keras_model.h5,30.344829
5,Colon,Colon_keras_model.h5,32.347140
6,Esophagus,Esophagus_keras_model.h5,34.867775
7,Heart,Heart_keras_model.h5,36.666667
8,Lung,Lung_keras_model.h5,32.552692
9,Muscle,Muscle_keras_model.h5,31.737590
10,Nerve,Nerve_keras_model.h5,32.125604
